In [1]:
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from os import urandom

# 1. Generate AES-128 key (16 bytes)
key = AESGCM.generate_key(bit_length=128)
aesgcm = AESGCM(key)

# 2. Generate a random 96-bit (12-byte) nonce (recommended size for GCM)
nonce = urandom(12)

# 3. Define plaintext and optional associated data (AAD)
plaintext = b"A un amigo perdido"
aad = b"authenticated-but-not-encrypted"

# 4. Encrypt
ciphertext = aesgcm.encrypt(nonce, plaintext, aad)

In [2]:
print("Key:", key.hex())
print("Nonce:", nonce.hex())



Key: 4724e60f55cbd868b9db49bb9e6afb6c
Nonce: 59c6ac49c2e0b63982dbac30


In [10]:
print("Plaintext (hex):", plaintext)

Plaintext (hex): b'A un amigo perdido'


In [11]:
print("Plaintext (hex):", plaintext.hex())

Plaintext (hex): 4120756e20616d69676f207065726469646f


In [12]:
print(' '.join(format(b, '08b') for b in plaintext))

01000001 00100000 01110101 01101110 00100000 01100001 01101101 01101001 01100111 01101111 00100000 01110000 01100101 01110010 01100100 01101001 01100100 01101111


In [13]:
print("Ciphertext (hex):", ciphertext.hex())

Ciphertext (hex): 368a3217e26d8d08cc2cbaf1dce222876450a3a70f2f61e3d3018084e8fe63c4a20f


In [14]:
print(' '.join(format(b, '08b') for b in ciphertext))

00110110 10001010 00110010 00010111 11100010 01101101 10001101 00001000 11001100 00101100 10111010 11110001 11011100 11100010 00100010 10000111 01100100 01010000 10100011 10100111 00001111 00101111 01100001 11100011 11010011 00000001 10000000 10000100 11101000 11111110 01100011 11000100 10100010 00001111


In [16]:

# -------------------
# 🔓 Decryption
# -------------------

# 5. Decrypt (will raise exception if tampered)
decrypted = aesgcm.decrypt(nonce, ciphertext, aad)
print("Decrypted:", decrypted.decode())

Decrypted: A un amigo perdido


In [2]:
# AES Round Demonstration (AES-128 Simplified)
# Shows SubBytes, ShiftRows, MixColumns, AddRoundKey for a single round

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os
import pprint

# --- Helper functions ---
def bytes_to_matrix(b):
    """Converts a 16-byte array into a 4x4 matrix."""
    return [list(b[i:i+4]) for i in range(0, 16, 4)]

def matrix_to_bytes(m):
    """Converts a 4x4 matrix into a 16-byte array."""
    return bytes(sum(m, []))

def xor_bytes(a, b):
    return bytes(i^j for i, j in zip(a, b))

# --- Basic AES S-Box ---   
#### Note that The prefix 0x of each entry just means "this is a hex number."####

########
########

S_BOX = [
    # 0     1      2      3     4      5      6      7     8      9      A      B     C      D      E      F
    0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76, # 0
    0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0, # 1
    0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15, # 2
    0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75, # 3
    0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84, # 4
    0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf, # 5
    0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8, # 6
    0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2, # 7
    0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73, # 8
    0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb, # 9
    0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79, # A
    0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08, # B
    0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, # C
    0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e, # D
    0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf, # E
    0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16  # F
]

def sub_bytes(state):
    return [[S_BOX[byte] for byte in row] for row in state]

def shift_rows(state):
    return [
        state[0],
        state[1][1:] + state[1][:1],
        state[2][2:] + state[2][:2],
        state[3][3:] + state[3][:3]
    ]

def add_round_key(state, key_matrix):
    return [[b ^ k for b, k in zip(row, key_row)] for row, key_row in zip(state, key_matrix)]

# --- Demo ---
plaintext = b"HelloAESRound1!"
key = os.urandom(16)

state = bytes_to_matrix(plaintext)
key_matrix = bytes_to_matrix(key)

print("Initial State:")
pprint.pprint(state)

# Initial AddRoundKey
state = add_round_key(state, key_matrix)
print("\nAfter AddRoundKey:")
pprint.pprint(state)

# SubBytes
state = sub_bytes(state)
print("\nAfter SubBytes:")
pprint.pprint(state)

# ShiftRows
state = shift_rows(state)
print("\nAfter ShiftRows:")
pprint.pprint(state)

# (MixColumns is omitted here for simplicity)
# You can add a simplified MixColumns later if desired

# Show final state
final_bytes = matrix_to_bytes(state)
print("\nFinal State as bytes:", final_bytes)
print("Final Hex:", final_bytes.hex())


Initial State:
[[72, 101, 108, 108], [111, 65, 69, 83], [82, 111, 117, 110], [100, 49, 33]]

After AddRoundKey:
[[96, 183, 214, 190], [187, 215, 181, 233], [115, 181, 70, 223], [77, 152, 44]]

After SubBytes:
[[208, 169, 246, 174], [234, 14, 213, 30], [143, 213, 90, 158], [227, 70, 113]]

After ShiftRows:
[[208, 169, 246, 174], [14, 213, 30, 234], [90, 158, 143, 213], [227, 70, 113]]

Final State as bytes: b'\xd0\xa9\xf6\xae\x0e\xd5\x1e\xeaZ\x9e\x8f\xd5\xe3Fq'
Final Hex: d0a9f6ae0ed51eea5a9e8fd5e34671
